# Recommender Systems with Surprise
- **Created by Andrés Segura Tinoco**
- **Created on May 23, 2019**

## Experiment description
- Model built from a plain text file
- The algorithm used is: KNNBasic
- Model trained using the technique of cross validation (5 folds)
- The RMSE and MAE metrics were used to estimate the model error
- Type of filtering: collaborative

In [1]:
# Load the Pandas libraries
import os

In [2]:
# Load Surprise libraries
from surprise import KNNBasic
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

## 1. Loading data

In [3]:
# Path to dataset file
file_path = os.path.expanduser('../data/u.data')

In [4]:
# Read the data into a Surprise dataset
reader = Reader(line_format = 'user item rating timestamp', sep = '\t', rating_scale = (1, 5))
data = Dataset.load_from_file(file_path, reader = reader)

## 2. Train the model and measure its error

In [5]:
# Use k-NN inspired algorithms
kk = 50
algo = KNNBasic(k = kk, verbose = True)

In [6]:
# Run 5-fold cross-validation and print results
cv = cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9804  0.9862  0.9805  0.9816  0.9773  0.9812  0.0029  
MAE (testset)     0.7724  0.7790  0.7770  0.7759  0.7729  0.7754  0.0025  
Fit time          0.62    0.64    0.77    0.67    0.70    0.68    0.05    
Test time         4.76    6.51    6.05    5.75    6.71    5.96    0.69    


## 3. Make predictions

In [7]:
# Without real rating
p1 = algo.predict(uid = '13', iid = '181', verbose = True)

user: 13         item: 181        r_ui = None   est = 4.32   {'actual_k': 50, 'was_impossible': False}


In [8]:
# With real rating
p2 = algo.predict(uid = '196', iid = '302', r_ui = 4, verbose = True)

user: 196        item: 302        r_ui = 4.00   est = 4.04   {'actual_k': 50, 'was_impossible': False}


---
<a href="https://ansegura7.github.io/RS_Surprise/">&laquo; Home</a>